## Pipenv

In [ ]:
#gunicorn
#lightgbm==3.3.1
#scikit-learn==1.0.2
#xgboost==1.4.2
#catboost==1.0.3
#Flask
#pandas and numpy

In [14]:
import pandas as pd
import numpy as np
import json
import pickle
from IPython.display import display

In [16]:
pd.set_option('display.max_columns', None)
import warnings 
warnings.filterwarnings('ignore')

In [6]:
data = pd.read_csv('C:\\Users\\user\\Solo Dolo\\NBA_rookie_career\\data\\NBA_Rookies_1979_2020(greater than 2 seasons).csv')
print(data.shape)
data.head()

(2842, 41)


,Unnamed: 0,Player,Team,Year,Conf,Age,Career,Games,MP,PTS,...,FTpg,ORBpg,DRBpg,TRBpg,ASTpg,STLpg,BLKpg,TOVpg,PFpg,Target
0,0,James Bailey,SEA,1979,WEST,22.0,9.0,67.0,726.0,312.0,...,1.01,1.06,1.88,2.9,0.4,0.31,0.81,1.18,1.73,1
1,1,Billy Ray Bates,POR,1980,WEST,23.0,4.0,16.0,235.0,180.0,...,1.75,0.81,1.00,1.8,1.9,0.88,0.12,1.25,1.62,1
2,2,Larry Bird*,BOS,1979,EAST,23.0,13.0,82.0,2955.0,1745.0,...,3.67,2.63,7.76,10.4,4.5,1.74,0.65,3.21,3.40,1
3,3,Lawrence Boston,WSB,1980,EAST,23.0,1.0,13.0,125.0,56.0,...,0.62,1.46,1.54,3.0,0.2,0.31,0.15,0.62,1.92,0
4,4,Dudley Bradley,IND,1979,EAST,22.0,9.0,82.0,2027.0,688.0,...,1.66,0.84,1.88,2.7,3.1,2.57,0.59,2.02,2.37,1


In [7]:
data[data.Year==2016]

,Unnamed: 0,Player,Team,Year,Conf,Age,Career,Games,MP,PTS,...,FTpg,ORBpg,DRBpg,TRBpg,ASTpg,STLpg,BLKpg,TOVpg,PFpg,Target
2458,2458,Thanasis Antetokounmpo,NYK,2016,EAST,23.0,4.0,2.0,6.0,6.0,...,0.00,0.00,0.50,0.5,0.0,0.00,0.00,0.00,1.00,1
2459,2459,Keith Appling,ORL,2016,EAST,23.0,1.0,5.0,27.0,6.0,...,0.40,0.00,0.20,0.2,0.2,0.20,0.00,0.60,1.00,0
2464,2464,Rakeem Christmas,IND,2016,EAST,24.0,2.0,1.0,6.0,4.0,...,0.00,1.00,0.00,1.0,0.0,0.00,0.00,0.00,1.00,0
2465,2465,Coty Clarke,BOS,2016,EAST,23.0,1.0,3.0,6.0,6.0,...,0.00,0.00,0.33,0.3,0.0,0.00,0.00,0.33,0.00,0
2468,2468,Bryce Dejean-Jones,NOP,2016,WEST,23.0,1.0,14.0,279.0,79.0,...,0.79,0.57,2.86,3.4,1.1,0.71,0.14,0.79,1.21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2613,2613,Kyle Wiltjer,HOU,2016,EAST,24.0,1.0,14.0,44.0,13.0,...,0.07,0.29,0.43,0.7,0.1,0.21,0.07,0.36,0.29,0
2614,2614,Stephen Zimmerman,ORL,2016,EAST,20.0,1.0,19.0,108.0,23.0,...,0.16,0.58,1.26,1.8,0.2,0.11,0.26,0.16,0.89,0
2615,2615,Paul Zipser,CHI,2016,EAST,22.0,2.0,44.0,843.0,240.0,...,0.70,0.34,2.50,2.8,0.8,0.34,0.36,0.91,1.77,0
2616,2616,Ivica Zubac,LAL,2016,WEST,19.0,6.0,38.0,609.0,284.0,...,0.84,1.08,3.11,4.2,0.8,0.37,0.87,0.79,1.74,1


In [2]:
predictors = ['MPpg', 'PTSpg', 'FGpg', '3Ppg', 'FTpg', 'ORBpg', 'DRBpg', 'TRBpg',
       'ASTpg', 'STLpg', 'BLKpg', 'TOVpg', 'PFpg', 'Age', 'Year_index',
       'Team_index', 'FT', 'FT%', 'Games', 'MP', 'FG%']

In [10]:
with open('team_mapper.json') as json_file:
    team_mapper = json.load(json_file)

In [5]:
team_mapper

{'ATL': 0,
 'BOS': 1,
 'BRK': 2,
 'CHH': 3,
 'CHI': 4,
 'CLE': 5,
 'DAL': 6,
 'DEN': 7,
 'DET': 8,
 'GSW': 9,
 'HOU': 10,
 'IND': 11,
 'LAC': 12,
 'LAL': 13,
 'MEM': 14,
 'MIA': 15,
 'MIL': 16,
 'MIN': 17,
 'NOP': 18,
 'NYK': 19,
 'OKC': 20,
 'ORL': 21,
 'PHI': 22,
 'PHO': 23,
 'POR': 24,
 'SAC': 25,
 'SAS': 26,
 'TOR': 27,
 'UTA': 28,
 'WSH': 29}

In [4]:
models ={}
with open('rookie-models2.bin', 'rb') as f_in:
    models['lr'],models['lgb'],models['xgb'],models['cats'],models['ada'],models['gbm'],models['rf'] = pickle.load(f_in)

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is

In [5]:
models

{'lr': LogisticRegression(C=0.1, solver='liblinear'),
 'lgb': LGBMClassifier(colsample_bytree=0.9, learning_rate=0.008, max_depth=5,
                metric='binary_logloss', min_child_samples=10, n_estimators=5000,
                num_leaves=120, random_state=34, reg_alpha=5, reg_lambda=2,
                subsample=0.7, subsample_freq=2),
 'xgb': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.01, max_delta_step=0, max_depth=5,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=5000, n_jobs=-1, num_parallel_tree=1,
               random_state=34, reg_alpha=10, reg_lambda=1, scale_pos_weight=1,
               subsample=0.7, tree_method='exact', validate_parameters=1,
               verbosity=None),
 'cats': <catboost.core.CatBoostClassifier at

In [8]:
def predict_single(feature_dict_list,models,predictors):
    #feature_dict['Year_index'] = feature_dict['Year'] - 1979
    #feature_dict['Team_index'] = team_mapper[feature_dict['Team']]
    df = pd.DataFrame.from_dict(feature_dict_list)
    df['Year_index'] = df['Year'] - 1979
    df['Team_index'] = df['Team'].map(team_mapper)
    final_predictions = (
         (models['lr'].predict_proba(df[predictors])[:,1]) +
        (models['lgb'].predict_proba(df[predictors])[:,1]) +
        (models['xgb'].predict_proba(df[predictors])[:,1]) +
        (models['cats'].predict_proba(df[predictors])[:,1])+ 
        (models['ada'].predict_proba(df[predictors])[:,1])+
         (models['gbm'].predict_proba(df[predictors])[:,1])+
         (models['rf'].predict_proba(df[predictors])[:,1])
    )/7
    final_preds = (final_predictions>=0.52)*1
    return final_predictions,final_preds

In [80]:
def predict_single(name,models,predictors):
    df = data[data.Player == name]
    display(df)
    #df['Year_index'] = df['Year'] - 1979
    #df['Team_index'] = df['Team'].map(team_mapper)
    final_predictions = (
         (models['lr'].predict_proba(df[predictors])[:,1]) +
        (models['lgb'].predict_proba(df[predictors])[:,1]) +
        (models['xgb'].predict_proba(df[predictors])[:,1]) +
        (models['cats'].predict_proba(df[predictors])[:,1])+ 
        (models['ada'].predict_proba(df[predictors])[:,1])+
         (models['gbm'].predict_proba(df[predictors])[:,1])+
         (models['rf'].predict_proba(df[predictors])[:,1])
    )/7
    final_preds = (final_predictions>=0.52)*1
    return final_predictions[0],final_preds[0]

In [10]:
l = [{'Unnamed: 0': 2458,
  'Player': 'Thanasis Antetokounmpo',
  'Team': 'NYK',
  'Year': 2016,
  'Conf': 'EAST',
  'Age': 23.0,
  'Career': 4.0,
  'Games': 2.0,
  'MP': 6.0,
  'PTS': 6.0,
  'FG': 3.0,
  'FGA': 4.0,
  'FG%': 0.75,
  '3P': 0.0,
  '3PA': 1.0,
  '3P%': 0.0,
  'FT': 0.0,
  'FTA': 0.0,
  'FT%': 0.0,
  'ORB': 0.0,
  'DRB': 1.0,
  'TRB': 1.0,
  'AST': 0.0,
  'STL': 0.0,
  'BLK': 0.0,
  'TOV': 0.0,
  'PF': 2.0,
  'MPpg': 3.0,
  'PTSpg': 3.0,
  'FGpg': 1.5,
  '3Ppg': 0.0,
  'FTpg': 0.0,
  'ORBpg': 0.0,
  'DRBpg': 0.5,
  'TRBpg': 0.5,
  'ASTpg': 0.0,
  'STLpg': 0.0,
  'BLKpg': 0.0,
  'TOVpg': 0.0,
  'PFpg': 1.0,
  'target>=2': 1,
  'target>=3': 1,
  'target>=4': 1,
  'target>=5': 0,
  'Team_index': 19,
  'Year_index': 37},
 {'Unnamed: 0': 2616,
  'Player': 'Ivica Zubac',
  'Team': 'LAL',
  'Year': 2016,
  'Conf': 'WEST',
  'Age': 19.0,
  'Career': 6.0,
  'Games': 38.0,
  'MP': 609.0,
  'PTS': 284.0,
  'FG': 126.0,
  'FGA': 238.0,
  'FG%': 0.529,
  '3P': 0.0,
  '3PA': 3.0,
  '3P%': 0.0,
  'FT': 32.0,
  'FTA': 49.0,
  'FT%': 0.653,
  'ORB': 41.0,
  'DRB': 118.0,
  'TRB': 159.0,
  'AST': 30.0,
  'STL': 14.0,
  'BLK': 33.0,
  'TOV': 30.0,
  'PF': 66.0,
  'MPpg': 16.03,
  'PTSpg': 7.5,
  'FGpg': 3.32,
  '3Ppg': 0.0,
  'FTpg': 0.84,
  'ORBpg': 1.08,
  'DRBpg': 3.11,
  'TRBpg': 4.2,
  'ASTpg': 0.8,
  'STLpg': 0.37,
  'BLKpg': 0.87,
  'TOVpg': 0.79,
  'PFpg': 1.74,
  'target>=2': 1,
  'target>=3': 1,
  'target>=4': 1,
  'target>=5': 1,
  'Team_index': 13,
  'Year_index': 37}]

In [11]:
predict_single(feature_dict_list=l,models=models,predictors=predictors)

(array([0.11916493, 0.87706843]), array([0, 1]))

In [81]:
predict_single(name='Chris Bosh*',models=models,predictors=predictors)

,Unnamed: 0,Player,Team,Year,Conf,Age,Career,Games,MP,PTS,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,MPpg,PTSpg,FGpg,3Ppg,FTpg,ORBpg,DRBpg,TRBpg,ASTpg,STLpg,BLKpg,TOVpg,PFpg,Target,Year_index,Team_index
1571,1593,Chris Bosh*,TOR,2003,EAST,19.0,13.0,75.0,2510.0,861.0,327.0,712.0,0.459,5.0,14.0,0.357,202.0,288.0,0.701,191.0,366.0,557.0,78.0,59.0,106.0,107.0,215.0,33.47,11.5,4.36,0.07,2.69,2.55,4.88,7.4,1.0,0.79,1.41,1.43,2.87,1,24,27


(0.9408083709096479, 1)

In [22]:
data_ro = pd.read_csv('C:\\Users\\user\\Solo Dolo\\NBA_rookie_career\\data\\nba_rookie_data_2021-2022.csv')
print(data_ro.shape)


(52, 21)


In [28]:
data_ro.head()

,player,games_played,minutes_played,points,field_goals_made,field_goals_attempted,field_goal_percentage,three_points_made,three_points_attempted,three_point_percentage,free_throws_made,free_throws_attempted,free_throw_percentage,offensive_rebounds,defensive_rebounds,rebounds,assists,steals,blocks,turnovers,efficiency
0,Cade Cunningham,63,32.5,17.5,6.7,16.1,41.8,1.8,5.7,31.8,2.2,2.6,84.3,0.9,4.6,5.5,5.5,1.2,0.7,3.6,17.0
1,Jalen Green,65,31.7,17.0,6.0,14.1,42.4,2.4,6.8,34.5,2.7,3.4,80.1,0.5,2.9,3.4,2.6,0.7,0.3,2.0,13.2
2,Scottie Barnes,71,35.5,15.5,6.2,12.7,49.1,0.8,2.6,30.1,2.2,3.0,73.2,2.7,4.9,7.6,3.5,1.1,0.8,1.8,19.3
3,Franz Wagner,79,30.7,15.2,5.8,12.3,46.8,1.2,3.4,35.6,2.4,2.8,86.3,1.1,3.4,4.5,2.9,0.9,0.4,1.5,15.4
4,Evan Mobley,67,33.9,14.9,6.1,12.0,50.6,0.3,1.4,25.3,2.4,3.6,66.4,2.0,6.2,8.2,2.5,0.8,1.6,2.0,19.0


In [26]:
data_ro_tm = pd.read_csv('C:\\Users\\user\\Solo Dolo\\NBA_rookie_career\\data\\player_team.csv')
print(data_ro_tm.shape)

(52, 5)


In [27]:
data_ro_tm.head()

,Unnamed: 0,index,player,position,team
0,0,1,Cade Cunningham,"PG,SG",DET
1,1,2,Evan Mobley,"PF,C",CLE
2,2,3,Scottie Barnes,"SF,PF",TOR
3,3,4,Herbert Jones,"SF,PF",NOP
4,4,5,Franz Wagner,"SG,SF,PF",ORL


In [32]:
rookie_2021 = data_ro.merge(data_ro_tm[['player','team','position']],on='player')
rookie_2021['age'] = 22
rookie_2021['year']

In [45]:
data_20_22[data_20_22.Player=='Cade Cunningham'][predictors]

,MPpg,PTSpg,FGpg,3Ppg,FTpg,ORBpg,DRBpg,TRBpg,ASTpg,STLpg,BLKpg,TOVpg,PFpg,Age,Year_index,Team_index,FT,FT%,Games,MP,FG%
233,32.6,17.4,6.703125,1.78125,2.21875,0.90625,4.59375,5.5,5.6,1.21875,0.671875,3.65625,3.125,20,42,8,142,0.845,64,2088,0.416


In [33]:
rookie_2021

,player,games_played,minutes_played,points,field_goals_made,field_goals_attempted,field_goal_percentage,three_points_made,three_points_attempted,three_point_percentage,free_throws_made,free_throws_attempted,free_throw_percentage,offensive_rebounds,defensive_rebounds,rebounds,assists,steals,blocks,turnovers,efficiency,team,position
0,Cade Cunningham,63,32.5,17.5,6.7,16.1,41.8,1.8,5.7,31.8,2.2,2.6,84.3,0.9,4.6,5.5,5.5,1.2,0.7,3.6,17.0,DET,"PG,SG"
1,Jalen Green,65,31.7,17.0,6.0,14.1,42.4,2.4,6.8,34.5,2.7,3.4,80.1,0.5,2.9,3.4,2.6,0.7,0.3,2.0,13.2,HOU,SG
2,Scottie Barnes,71,35.5,15.5,6.2,12.7,49.1,0.8,2.6,30.1,2.2,3.0,73.2,2.7,4.9,7.6,3.5,1.1,0.8,1.8,19.3,TOR,"SF,PF"
3,Franz Wagner,79,30.7,15.2,5.8,12.3,46.8,1.2,3.4,35.6,2.4,2.8,86.3,1.1,3.4,4.5,2.9,0.9,0.4,1.5,15.4,ORL,"SG,SF,PF"
4,Evan Mobley,67,33.9,14.9,6.1,12.0,50.6,0.3,1.4,25.3,2.4,3.6,66.4,2.0,6.2,8.2,2.5,0.8,1.6,2.0,19.0,CLE,"PF,C"
5,Chris Duarte,55,28.0,13.1,4.9,11.3,43.2,1.7,4.6,36.9,1.6,2.0,80.4,0.7,3.4,4.1,2.1,1.0,0.2,1.6,12.0,IND,"SG,SF"
6,Josh Giddey,54,31.5,12.5,5.2,12.4,41.9,1.0,3.9,26.3,1.0,1.5,70.9,1.8,6.0,7.8,6.4,0.9,0.4,3.2,17.2,OKL,"PG,SG,SF"
7,Jalen Suggs,47,27.5,11.9,4.2,11.5,36.2,0.9,4.1,21.4,2.7,3.5,77.3,0.6,3.0,3.6,4.4,1.2,0.4,3.0,10.3,ORL,"PG,SG"
8,Davion Mitchell,73,27.4,11.2,4.6,11.1,41.6,1.4,4.3,31.6,0.7,1.0,64.0,0.5,1.8,2.3,4.0,0.8,0.3,1.5,10.1,SAC,"PG,SG"
9,Tre Mann,60,22.8,10.4,3.9,9.8,39.3,1.6,4.5,36.0,1.1,1.4,79.3,0.4,2.4,2.8,1.5,0.8,0.2,1.2,8.4,OKL,"PG,SG"


In [34]:
data_20_22 = pd.read_csv('C:\\Users\\user\\Solo Dolo\\NBA_rookie_career\\data\\rookie_20-22.csv')

In [75]:
data_20_22.head()

,Unnamed: 0,Rk,Player,Debut,Age,Career,Games,MP,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%,MPpg,PTSpg,TRBpg,ASTpg,Team,Year,Year_index,Team_index,FGpg,3Ppg,FTpg,ORBpg,DRBpg,STLpg,BLKpg,TOVpg,PFpg
0,0,1,Kyle Alexander,"Aug 6, '20, MIA @ MIL",23,1,2,13,1,2,0,0,0,0,2,3,0,0,0,1,1,2,0.500,0.000,0.000,6.5,1.0,1.5,0.0,MIA,2020,41,15,0.500000,0.000000,0.000000,1.000000,0.500000,0.000000,0.000000,0.500000,0.500000
1,1,2,Nickeil Alexander-Walker,"Oct 22, '19, NOP @ TOR",21,3,158,3064,543,1404,227,690,154,212,59,415,347,110,53,216,248,1467,0.387,0.329,0.726,19.4,9.3,2.6,2.2,NOP,2019,40,18,3.436709,1.436709,0.974684,0.373418,2.226582,0.696203,0.335443,1.367089,1.569620
2,2,3,RJ Barrett,"Oct 23, '19, NYK @ SAS",19,3,198,6632,1246,2981,325,910,652,936,184,1101,568,151,53,414,452,3469,0.418,0.357,0.697,33.5,17.5,5.6,2.9,NYK,2019,40,19,6.292929,1.641414,3.292929,0.929293,4.670707,0.762626,0.267677,2.090909,2.282828
3,3,4,Darius Bazley,"Oct 23, '19, OKC @ UTA",19,3,185,4768,672,1656,212,696,285,410,152,1078,237,110,136,257,183,1841,0.406,0.305,0.695,25.8,10.0,5.8,1.3,OKC,2019,40,20,3.632432,1.145946,1.540541,0.821622,4.978378,0.594595,0.735135,1.389189,0.989189
4,4,5,Goga Bitadze,"Oct 26, '19, IND @ CLE",20,3,149,1761,284,596,51,201,133,188,164,433,115,40,139,91,266,752,0.477,0.254,0.707,11.8,5.0,2.9,0.8,IND,2019,40,11,1.906040,0.342282,0.892617,1.100671,1.799329,0.268456,0.932886,0.610738,1.785235


In [38]:
data_20_22.rename(columns = {'Yrs':'Career'},inplace=True)

In [43]:
data.head()

,Unnamed: 0,Player,Team,Year,Conf,Age,Career,Games,MP,PTS,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,MPpg,PTSpg,FGpg,3Ppg,FTpg,ORBpg,DRBpg,TRBpg,ASTpg,STLpg,BLKpg,TOVpg,PFpg,Target
0,0,James Bailey,SEA,1979,WEST,22.0,9.0,67.0,726.0,312.0,122.0,271.0,0.450,0.0,0.0,0.000,68.0,101.0,0.673,71.0,126.0,197.0,28.0,21.0,54.0,79.0,116.0,10.84,4.7,1.82,0.00,1.01,1.06,1.88,2.9,0.4,0.31,0.81,1.18,1.73,1
1,1,Billy Ray Bates,POR,1980,WEST,23.0,4.0,16.0,235.0,180.0,72.0,146.0,0.493,8.0,19.0,0.421,28.0,39.0,0.718,13.0,16.0,29.0,31.0,14.0,2.0,20.0,26.0,14.69,11.3,4.50,0.50,1.75,0.81,1.00,1.8,1.9,0.88,0.12,1.25,1.62,1
2,2,Larry Bird*,BOS,1979,EAST,23.0,13.0,82.0,2955.0,1745.0,693.0,1463.0,0.474,58.0,143.0,0.406,301.0,360.0,0.836,216.0,636.0,852.0,370.0,143.0,53.0,263.0,279.0,36.04,21.3,8.45,0.71,3.67,2.63,7.76,10.4,4.5,1.74,0.65,3.21,3.40,1
3,3,Lawrence Boston,WSB,1980,EAST,23.0,1.0,13.0,125.0,56.0,24.0,52.0,0.462,0.0,0.0,0.000,8.0,13.0,0.615,19.0,20.0,39.0,2.0,4.0,2.0,8.0,25.0,9.62,4.3,1.85,0.00,0.62,1.46,1.54,3.0,0.2,0.31,0.15,0.62,1.92,0
4,4,Dudley Bradley,IND,1979,EAST,22.0,9.0,82.0,2027.0,688.0,275.0,609.0,0.452,2.0,5.0,0.400,136.0,174.0,0.782,69.0,154.0,223.0,252.0,211.0,48.0,166.0,194.0,24.72,8.4,3.35,0.02,1.66,0.84,1.88,2.7,3.1,2.57,0.59,2.02,2.37,1


In [40]:
dups=data.Player.value_counts()[data.Player.value_counts()>1].index

In [54]:
data = data[~data.Player.isin(dups)].reset_index(drop=True)

In [55]:
team_map ={
    'NOK':'NOP',#pelicans
    'SDC':'LAC',# clippers
    #'NOP':'NOP',#pelicans
    'CHO':'CHH', #hornets
    'KCK':'SAC', # kings
    'VAN':'MEM', # grizzs
    'NOH':'NOP', # pelicans
    #'BRK':'BRK', # Nets
    'CHA':'CHH', # hornets
    #'OKC':'OKC', # okc
    'WAS':'WSH', #wizards
    #'TOR':'TOR', # raptors
    'WSB':'WSH', # wizards
    'SEA':'OKC', # sonics now okc
    'NJN':'BRK',#nets
    }
for old_team, new_team in team_map.items():
    data['Team'].replace(old_team, new_team, inplace=True)

In [56]:
    data['Year_index'] = data['Year'] - 1979
    data['Team_index'] = data['Team'].map(team_mapper)

In [51]:
for_merge = ['Player','Team','Year','Career'] + predictors 

In [60]:
all_data = pd.concat([data[for_merge],data_20_22[for_merge]])

In [63]:
all_data.shape

(3142, 25)

In [77]:
all_data.tail()

,Player,Team,Year,Career,MPpg,PTSpg,FGpg,3Ppg,FTpg,ORBpg,DRBpg,TRBpg,ASTpg,STLpg,BLKpg,TOVpg,PFpg,Age,Year_index,Team_index,FT,FT%,Games,MP,FG%
333,Ziaire Williams,MEM,2021,1.0,21.7,8.1,3.080645,1.225806,0.693548,0.419355,1.680645,2.1,1.0,0.564516,0.193548,0.661290,1.822581,20.0,42,14,43.0,0.782,62.0,1346.0,0.450
334,McKinley Wright IV,MIN,2021,1.0,3.8,1.0,0.400000,0.200000,0.000000,0.000000,0.000000,0.0,0.6,0.000000,0.000000,0.200000,0.200000,23.0,42,17,0.0,0.000,5.0,19.0,0.667
335,Moses Wright,LAC,2021,1.0,3.5,1.3,0.250000,0.000000,0.750000,0.250000,0.550000,0.8,0.5,0.000000,0.250000,0.000000,0.000000,23.0,42,12,3.0,1.000,4.0,14.0,0.250
336,Gabe York,IND,2022,1.0,10.5,4.0,1.000000,0.500000,1.500000,0.000000,1.000000,1.0,2.0,1.000000,0.500000,0.500000,2.000000,28.0,43,11,3.0,0.600,2.0,21.0,0.286
337,Omer Yurtseven,MIA,2021,1.0,12.6,5.3,2.321429,0.017857,0.678571,1.517857,3.782143,5.3,0.9,0.303571,0.357143,0.732143,1.500000,23.0,42,15,38.0,0.623,56.0,706.0,0.526


In [65]:
all_data.Year.value_counts()

2021    146
2019    119
1999    118
2018    106
2017    105
2012     98
2016     90
2020     85
1989     84
2005     82
2014     81
2006     78
1980     77
2015     74
1990     74
1988     73
1997     72
2001     72
1996     71
2004     71
1993     69
2007     67
2013     66
2002     66
2010     66
1995     65
1992     65
1986     64
1981     63
2008     63
1994     63
2000     63
2003     60
1984     60
1982     60
2011     58
1983     58
1991     58
2009     56
1987     54
1985     54
1979     47
2022     15
1998      6
Name: Year, dtype: int64

In [66]:
all_data.Player.value_counts()[all_data.Player.value_counts()>1]

Brandon Williams    2
Name: Player, dtype: int64

In [70]:
all_data.loc[(all_data.Player=='Brandon Williams') & (all_data.Year==1998),'Player'] = 'Brandon Williams (1998)'

In [78]:
all_data.to_csv('C:\\Users\\user\\Solo Dolo\\NBA_rookie_career\\data\\all_data_clean.csv',index=False)

In [79]:
all_data.Player.unique()

array(['James Bailey', 'Billy Ray Bates', 'Larry Bird*', ...,
       'Moses Wright', 'Gabe York', 'Omer Yurtseven'], dtype=object)